# Model training and hyperparameter tuning

In [ ]:
import pandas as pd
from ..src.data.make_dataset import data_preparation
from ..src.features.build_features import cbk_feature_engineering


## 1. Dataset

In [ ]:
train = pd.read_excel('../data/raw/Missão_Stone_-_Dados_de_trx_(3).xlsx', engine='openpyxl', sheet_name='Aba 1')

In [ ]:
test = pd.read_excel('../data/raw/Missão_Stone_-_Dados_de_trx_(3).xlsx', engine='openpyxl', sheet_name='Aba 2')

#### 1.1 Data prep

Nota: Foram criados utils em src para reproduzir todas as transformações e feature engineering performadas em notebook por reprodutibilidade

In [ ]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
def cbk_data_prep(df):
    df = data_preparation(df)
    df = cbk_feature_engineering(df)
    df = 

In [ ]:
dfs = [train,test]
for df in dfs:
    df = 